In [1]:
!ls "input/1850/nypl_1850_51_split_v2.txt"

input/1850/nypl_1850_51_split_v2.txt


In [2]:
#load data
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "input/1850/nypl_1850_51_split_v2.txt"
with open(file_path) as f:
  data = f.readlines()  

In [3]:
import pandas as pd
pd.DataFrame(data).shape

(47603, 1)

There are 47,603 rows

In [4]:
data

['NAMES TOO\n',
 'Ackley Addison, clerk, 47 Broadway, h. 39 Pearl\u2028Adams William, weigher, 20 Washington & 13 West,\u2028h. 170 Clinton, Brooklyn\n',
 'Aitkin Thomas, stonecutter, 601 Washington\n',
 'Albro Benjamin sen. 328 Bowery\n',
 'Allen E. W. publisher, 42 Orange\n',
 'Anderson Frederick 11. com. mer. 13 Front, h. 77\u2028Barrow\n',
 'Anderson John R. seaman, 586 E. 4th\n',
 'Anthes Peter, barber 34 Av. C\n',
 'Anthes & Eckardt, barbers, 34 Av. C\n',
 'Appell John, tailor, 40 Av. C\n',
 'Arcularius Jacob S. h. 175 Canal\n',
 'Arenfred George, 137 Norfolk\n',
 'Armstrong George, shoemaker, 53 Av. D\u2028Armstrong John D. coachmaker, 5 Ludlow\n',
 'Arnold Henry, drygoods, 1 Av. D, h. r. 1 Av. D\u2028Ash William A. saloon, 413 B.way, h. Morrisania\u2028Auchincloss John T. com. mer. 192 Fulton, h. 7\n',
 'Murray\n',
 'Auchincloss & Croxon, com. mers. 192 Fulton\u2028Bagley William, clerk, bank, 50 Wall, h. 91 Henry,\n',
 'Brooklyn\n',
 'BAIN ALEXANDER, telegraph inventor, 181\u2

In [5]:
# Replace Unicode \u2028 with \n
data = [entry.replace('\u2028', '\n') for entry in data]

In [6]:
# Save file
with open("input/1850/nypl_1850_51_split_v3.txt", 'w') as f:
    f.writelines("%s\n" % line for line in data)

In [7]:
!ls "input/1850/nypl_1850_51_split_v3.txt"

input/1850/nypl_1850_51_split_v3.txt


In [8]:
#load data
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "input/1850/nypl_1850_51_split_v3.txt"
with open(file_path) as f:
  data = f.readlines()  

In [9]:
import pandas as pd
pd.DataFrame(data).shape

(139296, 1)

There are now 139,295 rows

## Divide records into different sections based on first letter

In [10]:
data[0] = 'A.'
import re

d = {}
value = []
i = 0
for line in data:
  line = line.strip()
  if line:
    if len(line) < 3: 
      # 1850 data is not split by '***' instead it is split by 'B., C., D.'
      if bool(re.match("([A-Z]+\.)", line)) == True:
        # save previous value
        if value:
          if value[0] == "'":
            value = value[1:]
          d[key] = value
        
        key = line
        value = []
    else:
      value.append(line)

In [11]:
d.items

<function dict.items>

## Combline lines that belong to the same record

Criteria:

1) First letter of next line not equal to the initial letter in this section

2) Special cases in the start of next line: B’klyn, B'way, E., W.

In [12]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0
  
  while i < n:
    if i < n-1:
      if (value[i+1][0] != initial and value[i+1][0] != "*") or (value[i+1][0:2] in ["B’", "E.", "W."]) or (value[i+1][0:2] == ". "):
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value      

In [13]:
# remove "(see adv...."
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    n2 = len(ls)
    for j in range(n2):
      if "see" in ls[j]:
        value[i] = ' '.join(ls[:j])
        break

## Identify business vs person records

Made an update on 27/03/20 to change the index notation.

* The first letter of Company names were being removed by the function
    * ZUCCA BROTHERS
    * ARNAULT PETER
    * APPELL JACOB
    * APPLETON’S AMERICAN CYCLOPG5DIA
    * BURTON A. B 

In [14]:
# add an index to each record
# in another dictionary, for each index specify whether it's business or person record
index = 1
d_bp = []
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    if ls:
      if ls[0][0] == "*" or (len(ls[0]) > 1 and ls[0].isupper()): # Upper case as well
        value[i] = value[i][0:] # Changed from [0:] to [1:]
        d_bp.append([str(index), "b"])
      else:
        d_bp.append([str(index), "p"])
    
      value[i] = str(index) + "\t" + value[i]
      index += 1

## Save files

In [15]:
file = open("input/1850/nypl_1850_51_starred_clean.txt", "w")
for key,value in d.items():
  file.write(key + '\n')
  for record in value:
    file.write(record + '\n')
  file.write('\n')

file.close()

file2 = open("input/1850/index_business_person.txt", "w")
for item in d_bp:
  file2.write(item[0] + ' ' + item[1] + '\n')
  
file2.close()